In [10]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [11]:

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"]=os.getenv("LANGSMITH__PROJECT")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


In [13]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-4o")
print(llm)

client=<openai.resources.chat.completions.Completions object at 0x000001D13097EC00> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001D1309A4170> root_client=<openai.OpenAI object at 0x000001D130977F50> root_async_client=<openai.AsyncOpenAI object at 0x000001D13097D430> model_name='gpt-4o' model_kwargs={} openai_api_key=SecretStr('**********')


prompt 1

In [14]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based on the provided context.
Think step by step before providing a detailed answer.
I will tip you $100000 if the user finds the answer helpful.
<context>
{context}
</context>                                                                                                                                                                
Question: {input}
""")

prompt 2

In [15]:
from langchain_core.prompts import ChatPromptTemplate
prompt_2=ChatPromptTemplate.from_messages(
    [
        ("system","You are an expert Data Scientist and Gen AI Engineer. Provide me answers based on the asked question "),
        ("user","{input}")

    ]
)
prompt_2

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert Data Scientist and Gen AI Engineer. Provide me answers based on the asked question '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

## Chain

In [16]:
from langchain_core.output_parsers import StrOutputParser
output_parser=StrOutputParser()
chain=prompt_2|llm|output_parser

response=chain.invoke({"input":"Can you tell me about Langsmith?"})
print(response)

LangSmith is a platform and set of tools provided by LangChain that focus on enhancing the development and management of applications powered by large language models (LLMs). Here are some key aspects of LangSmith:

1. **Development Workflow**: LangSmith assists developers throughout the entire lifecycle of creating applications with LLMs. This includes design, testing, and optimization, providing a seamless integration with LangChain’s framework for building applications.

2. **Evaluation Metrics**: One of the main features of LangSmith is its emphasis on evaluating language model outputs. It offers tools for establishing evaluation workflows, allowing developers to create and utilize metrics that are crucial for assessing the performance and reliability of deployed models.

3. **Monitoring and Debugging**: LangSmith provides real-time monitoring and debugging capabilities. These tools help developers identify and address issues quickly, ensuring that the applications maintain high pe

In [17]:
'''from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
output_parser=JsonOutputParser()
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)'''

'from langchain_core.prompts import PromptTemplate\nfrom langchain_core.output_parsers import JsonOutputParser\noutput_parser=JsonOutputParser()\nprompt = PromptTemplate(\n    template="Answer the user query.\n{format_instructions}\n{query}\n",\n    input_variables=["query"],\n    partial_variables={"format_instructions": output_parser.get_format_instructions()},\n)'

In [18]:
'''chain = prompt_2|llm|output_parser

response=chain.invoke({"query":"Can you tell me about Langsmith?"})
print(response)'''

'chain = prompt_2|llm|output_parser\n\nresponse=chain.invoke({"query":"Can you tell me about Langsmith?"})\nprint(response)'

home_work = output parser, langchain python doc

## Document Chain

In [19]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)

## Data Ingestion

In [20]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader = WebBaseLoader("https://www.credo.ai/blog/cutting-through-the-noise-what-is-ai-governance", bs_kwargs={
        "parse_only": bs4.SoupStrainer(class_="text-rich-text w-richtext"),
    }
)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [21]:
doc = loader.load()
document = doc[0].page_content

## Splitting

In [22]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_text(document)

## Embedding

In [23]:
# from langchain_openai import OpenAIEmbeddings
# embedding = OpenAIEmbeddings()

Hugging_face Embedding

In [24]:
from langchain_huggingface import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\dp971\Coading\GEN_AI\AgenticAI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
em = embedding.embed_query("hi")
len(em)

384

## Vector store

In [26]:
from langchain_community.vectorstores import FAISS
db = FAISS.from_texts(texts=texts, embedding=embedding)

## Retriever

In [27]:
retriever = db.as_retriever(search_type="mmr")

## Retriever Chain

### User Query

In [28]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [29]:
response = retrieval_chain.invoke({"input": "what is Ai Governance?"})

In [30]:
response['answer']

"AI Governance refers to the framework of policies, processes, and practices that guide the development, deployment, and management of AI systems, with the aim of maximizing benefits and minimizing risks. Here’s a step-by-step explanation to better understand AI Governance based on the provided context:\n\n1. **Definition and Purpose**: \n   - AI Governance is about establishing a set of rules and procedures to oversee AI systems. It aims to ensure that AI is used responsibly, effectively, and fairly while managing the associated risks.\n\n2. **Core Elements**:\n   - **Oversight and Guardrails**: Governance acts as a supervisory mechanism, providing necessary checks and balances to reduce risk and ensure AI systems operate within acceptable parameters.\n   - **Stakeholders**: A range of individuals and entities have an interest in AI systems, including developers, policymakers, and end-users. Each has a role in governance and ensuring systems are aligned with societal norms and legal r